# Reorganizing the system matrices from  (Lin, Parker, 1999)

The objective is to describe how the system matrices presented in [1] were reorganized, in order to become more suitable to model multi-stage gearboxes with planetary and parallel stages. The reasons for reorganizing them are:
* the ring gear is assumed to be fixed, so its degrees-of-freedom should be removed from the system matrices
* these matrices will be combined for different stages in a finite-element manner, that would be easier if the state vector for each element were ordered from input to output elements, which in our case are the planet carrier and the sun gear, with the  $N$ planet gears and ring gear in between.

##### References:
[1] Lin, J., & Parker, R. G. (1999). Analytical characterization of the unique properties of planetary gear free vibration. Journal of Vibration and Acoustics, Transactions of the ASME, 121(3), 316–321. http://doi.org/10.1115/1.2893982 \
[2] Cooley, C. G., & Parker, R. G. (2012). Vibration Properties of High-Speed Planetary Gears With Gyroscopic Effects. Journal of Vibration and Acoustics, 134(6). http://doi.org/10.1115/1.4006646

In [1]:
from sympy import *
init_printing()

def symb(x, y, z = ''):
    return symbols('{0}_{1}{2}'.format(x, y, z), type = float)

## Displacement vector

The rotational coordinates are represented indirectly in the following way:
$$u_j = r_j \theta_j,$$
where $r_j$ is the base radius of sun, ring, and planets gears or the center distance for the carrier

In [2]:
N = 3 # number of planet gears
N_DOF = 3*(N + 3) # number of degrees of freedom
#            carrier,  ring,  sun + planets
idx_list = ['c'     , 'r'  , 's'] + list(range(1, N + 1))

q = Matrix([symb('q', v) for v in idx_list])
q_ = lambda idx: Matrix(symbols('x_{0} y_{0} u_{0}'.format(idx)))
q, q_('j')

⎛⎡q_c⎤       ⎞
⎜⎢   ⎥       ⎟
⎜⎢qᵣ ⎥       ⎟
⎜⎢   ⎥  ⎡x_j⎤⎟
⎜⎢qₛ ⎥  ⎢   ⎥⎟
⎜⎢   ⎥, ⎢y_j⎥⎟
⎜⎢q₁ ⎥  ⎢   ⎥⎟
⎜⎢   ⎥  ⎣u_j⎦⎟
⎜⎢q₂ ⎥       ⎟
⎜⎢   ⎥       ⎟
⎝⎣q₃ ⎦       ⎠

## Inertia matrix

First one needs to write the system matrices, defined in the Appendix of [1]. The inertia matrix can be seen below:
\begin{align}
    \mathbf{M} = \mathrm{diag}(\mathbf{M}_c, \mathbf{M}_r, \mathbf{M}_s, \mathbf{M}_1, \ldots, \mathbf{M}_N) \\
    \mathbf{M}_j = \mathrm{diag}(m_j, m_j, I_j/r_j^2), \quad j = c, r, s, 1, \ldots, N
\end{align}
where:
* $N$ is the number of planets
* $m_j$ is the mass of an element (sun, planet, ring, carrier)
* $I_j$ is the mass moment of inertia of an element
* $r_j$ is the base radius for sun, ring, and planet gears or the center distance for the carrier

In [3]:
M  = diag(*[symb('M', v) for v in idx_list])
M_ = lambda j: diag(symb('m', j), symb('m', j), symb('I', j)/symb('r', j)**2)

M, M_('j')

⎛⎡M_c  0   0   0   0   0 ⎤                  ⎞
⎜⎢                       ⎥  ⎡m_j   0    0  ⎤⎟
⎜⎢ 0   Mᵣ  0   0   0   0 ⎥  ⎢              ⎥⎟
⎜⎢                       ⎥  ⎢ 0   m_j   0  ⎥⎟
⎜⎢ 0   0   Mₛ  0   0   0 ⎥  ⎢              ⎥⎟
⎜⎢                       ⎥, ⎢          I_j ⎥⎟
⎜⎢ 0   0   0   M₁  0   0 ⎥  ⎢ 0    0   ────⎥⎟
⎜⎢                       ⎥  ⎢             2⎥⎟
⎜⎢ 0   0   0   0   M₂  0 ⎥  ⎣          r_j ⎦⎟
⎜⎢                       ⎥                  ⎟
⎝⎣ 0   0   0   0   0   M₃⎦                  ⎠

## Gyroscopic matrix:

In [4]:
G  = diag(*[symb('G', v) for v in idx_list])
G_ = lambda j: Matrix(3, 3, [0, -2*symb('m', j), 0, 2*symb('m', j), 0, 0, 0, 0, 0])

G, G_('j')

⎛⎡G_c  0   0   0   0   0 ⎤                    ⎞
⎜⎢                       ⎥                    ⎟
⎜⎢ 0   Gᵣ  0   0   0   0 ⎥                    ⎟
⎜⎢                       ⎥  ⎡  0    -2⋅m_j  0⎤⎟
⎜⎢ 0   0   Gₛ  0   0   0 ⎥  ⎢                ⎥⎟
⎜⎢                       ⎥, ⎢2⋅m_j    0     0⎥⎟
⎜⎢ 0   0   0   G₁  0   0 ⎥  ⎢                ⎥⎟
⎜⎢                       ⎥  ⎣  0      0     0⎦⎟
⎜⎢ 0   0   0   0   G₂  0 ⎥                    ⎟
⎜⎢                       ⎥                    ⎟
⎝⎣ 0   0   0   0   0   G₃⎦                    ⎠

## Stiffness matrix

### Centripetal component

In [5]:
K_Omega  = diag(*[symb('K', 'Omega', v) for v in idx_list])
K_Omega_ = lambda j: diag(symb('m', j), symb('m', j), 0)

K_Omega, K_Omega_('j')

⎛⎡K_Omegac     0         0         0         0         0    ⎤               ⎞
⎜⎢                                                          ⎥               ⎟
⎜⎢   0      K_Omegar     0         0         0         0    ⎥               ⎟
⎜⎢                                                          ⎥  ⎡m_j   0   0⎤⎟
⎜⎢   0         0      K_Omegas     0         0         0    ⎥  ⎢           ⎥⎟
⎜⎢                                                          ⎥, ⎢ 0   m_j  0⎥⎟
⎜⎢   0         0         0      K_Omega1     0         0    ⎥  ⎢           ⎥⎟
⎜⎢                                                          ⎥  ⎣ 0    0   0⎦⎟
⎜⎢   0         0         0         0      K_Omega2     0    ⎥               ⎟
⎜⎢                                                          ⎥               ⎟
⎝⎣   0         0         0         0         0      K_Omega3⎦               ⎠

### Bearing component:

In [6]:
func = lambda x: symb('K', x, 'b') if(not isinstance(x, int)) else 0

K_b  = diag(*[func(v) for v in idx_list])
K_b_ = lambda idx: diag(symb('k', idx, 'x'), symb('k', idx, 'y'), symb('k', idx, 'u'))
K_b, K_b_('j')

⎛⎡K_cb   0     0    0  0  0⎤                    ⎞
⎜⎢                         ⎥                    ⎟
⎜⎢ 0    K_rb   0    0  0  0⎥                    ⎟
⎜⎢                         ⎥  ⎡k_jx   0     0  ⎤⎟
⎜⎢ 0     0    K_sb  0  0  0⎥  ⎢                ⎥⎟
⎜⎢                         ⎥, ⎢ 0    k_jy   0  ⎥⎟
⎜⎢ 0     0     0    0  0  0⎥  ⎢                ⎥⎟
⎜⎢                         ⎥  ⎣ 0     0    k_ju⎦⎟
⎜⎢ 0     0     0    0  0  0⎥                    ⎟
⎜⎢                         ⎥                    ⎟
⎝⎣ 0     0     0    0  0  0⎦                    ⎠

where $k_{ix}$ and $k_{iy}$ are the bearing stiffness components in the $x$ and $y$ directions. The torsional components of the bearing stiffness are null, i.e. $k_{iu} = 0$, for $i = c, r, s$. The stiffness of the planet bearings can be found on the sub-matrices $K_c$ of the mesh stiffness matrix.
### Mesh component:
The mesh stiffness matrix is defined in the figure below, where it is sub-divided in blocks with colors, red, grey, and blue, called diagonal 1, diagonal 2, and off-diagonal, respectively.
\begin{align}
    \begin{bmatrix}
       \sum \mathbf{K}_{c1}^n & \mathbf{0}             & \mathbf{0}             & \mathbf{K}_{c2}^1 & \mathbf{K}_{c2}^2 & \ldots & \mathbf{K}_{c2}^N\\
                              & \sum \mathbf{K}_{r1}^n & \mathbf{0}             & \mathbf{K}_{r2}^1 & \mathbf{K}_{r2}^2 & \ldots & \mathbf{K}_{r2}^N\\
                              &                        & \sum \mathbf{K}_{s1}^n & \mathbf{K}_{s2}^1 & \mathbf{K}_{s2}^2 & \ldots & \mathbf{K}_{s2}^N \\
                              &                        &                        & \mathbf{K}_{pp}^1 & \mathbf{0}       & \ldots & \mathbf{0} \\
       {\rm symmetric}        &                        &                        &                   & \ddots           & \ldots & \vdots \\
                              &                        &                        &                   &                  &        & \mathbf{K}_{pp}^N
    \end{bmatrix}
\end{align}
![alt text](K_m_LP_99.png)

#### Diagonal 1

In [7]:
K_d1 = diag(Sum(symb('K', 'c1', '^j'), (symbols('j'), 1, N)),
            Sum(symb('K', 'r1', '^j'), (symbols('j'), 1, N)),
            Sum(symb('K', 's1', '^j'), (symbols('j'), 1, N)))
K_d1

⎡  3                                        ⎤
⎢ ___                                       ⎥
⎢ ╲                                         ⎥
⎢  ╲                                        ⎥
⎢  ╱   K_c1__j        0              0      ⎥
⎢ ╱                                         ⎥
⎢ ‾‾‾                                       ⎥
⎢j = 1                                      ⎥
⎢                                           ⎥
⎢                 3                         ⎥
⎢                ___                        ⎥
⎢                ╲                          ⎥
⎢                 ╲                         ⎥
⎢      0          ╱   K_r1__j        0      ⎥
⎢                ╱                          ⎥
⎢                ‾‾‾                        ⎥
⎢               j = 1                       ⎥
⎢                                           ⎥
⎢                                3          ⎥
⎢                               ___         ⎥
⎢                               ╲           ⎥
⎢                                ╲

#### Diagonal 2

In [8]:
K_d2 = diag(*[symb('K', 'pp', '^{}'.format(v + 1)) for v in range(N)])
K_d2

⎡K¹ₚₚ   0     0  ⎤
⎢                ⎥
⎢ 0    K²ₚₚ   0  ⎥
⎢                ⎥
⎣ 0     0    K³ₚₚ⎦

#### Off-diagonal

In [9]:
K_od = Matrix([[symb('K', 'c2', '^{}'.format(v + 1)) for v in range(N)], 
               [symb('K', 'r2', '^{}'.format(v + 1)) for v in range(N)], 
               [symb('K', 's2', '^{}'.format(v + 1)) for v in range(N)]])
K_od

⎡K_c2__1  K_c2__2  K_c2__3⎤
⎢                         ⎥
⎢ K¹ᵣ₂     K²ᵣ₂     K³ᵣ₂  ⎥
⎢                         ⎥
⎣ K¹ₛ₂     K²ₛ₂     K³ₛ₂  ⎦

#### Assembly

In [10]:
K_m = BlockMatrix([[          K_d1 , K_od],
                   [transpose(K_od), K_d2]])
K_m = Matrix(K_m)

if(not K_m.is_symmetric()):
    print('error.')
K_m

⎡  3                                                                   ⎤
⎢ ___                                                                  ⎥
⎢ ╲                                                                    ⎥
⎢  ╲                                                                   ⎥
⎢  ╱   K_c1__j        0              0        K_c2__1  K_c2__2  K_c2__3⎥
⎢ ╱                                                                    ⎥
⎢ ‾‾‾                                                                  ⎥
⎢j = 1                                                                 ⎥
⎢                                                                      ⎥
⎢                 3                                                    ⎥
⎢                ___                                                   ⎥
⎢                ╲                                                     ⎥
⎢                 ╲                                                    ⎥
⎢      0          ╱   K_r1__j        0         K¹ᵣ₂

The submatrices of $\mathbf{K}_m$ will be defined later on, after we reorganize the overall system matrices.
## Reorganizing the system matrices:
The following steps will be implemented using matrices that should be pre and post-multiply the system matrices and submatrices in the following way:
\begin{align}
    \mathbf{A}_\mathrm{new} = \mathbf{Y}^T \mathbf{A}_\mathrm{old} \mathbf{Y}
\end{align}

### Eliminating the ring element:
One can do that using linear algebra with the matrix $\mathbf{R}$ described as follows:

In [11]:
N_eo = N + 3 # number of elements (original)
N_e  = N_eo - 1 # number of elements (new)

W = eye(N_e, N_eo)
for idx in range(N_e - 1):
    c1 = N_e - idx
    c2 = c1 - 1
    W = W.elementary_col_op(op = 'n<->m', col1 = c1, col2 = c2)
# q_2 = W * q:
W*q, W, q

⎛                           ⎡q_c⎤⎞
⎜⎡q_c⎤  ⎡1  0  0  0  0  0⎤  ⎢   ⎥⎟
⎜⎢   ⎥  ⎢                ⎥  ⎢qᵣ ⎥⎟
⎜⎢qₛ ⎥  ⎢0  0  1  0  0  0⎥  ⎢   ⎥⎟
⎜⎢   ⎥  ⎢                ⎥  ⎢qₛ ⎥⎟
⎜⎢q₁ ⎥, ⎢0  0  0  1  0  0⎥, ⎢   ⎥⎟
⎜⎢   ⎥  ⎢                ⎥  ⎢q₁ ⎥⎟
⎜⎢q₂ ⎥  ⎢0  0  0  0  1  0⎥  ⎢   ⎥⎟
⎜⎢   ⎥  ⎢                ⎥  ⎢q₂ ⎥⎟
⎜⎣q₃ ⎦  ⎣0  0  0  0  0  1⎦  ⎢   ⎥⎟
⎝                           ⎣q₃ ⎦⎠

### Changing the element order:
One can do that with the matrix $\mathbf{V}$, which is defined as: 

In [12]:
N1 = N_e - 1

V = zeros(N_e)
V[0   , 0 ] = 1
V[  N1, 1 ] = 1
V[1:N1, 2:] = eye(N)

# q_3 = V * q_2
W*q, V, V*W*q

⎛⎡q_c⎤  ⎡1  0  0  0  0⎤  ⎡q_c⎤⎞
⎜⎢   ⎥  ⎢             ⎥  ⎢   ⎥⎟
⎜⎢qₛ ⎥  ⎢0  0  1  0  0⎥  ⎢q₁ ⎥⎟
⎜⎢   ⎥  ⎢             ⎥  ⎢   ⎥⎟
⎜⎢q₁ ⎥, ⎢0  0  0  1  0⎥, ⎢q₂ ⎥⎟
⎜⎢   ⎥  ⎢             ⎥  ⎢   ⎥⎟
⎜⎢q₂ ⎥  ⎢0  0  0  0  1⎥  ⎢q₃ ⎥⎟
⎜⎢   ⎥  ⎢             ⎥  ⎢   ⎥⎟
⎝⎣q₃ ⎦  ⎣0  1  0  0  0⎦  ⎣qₛ ⎦⎠

Both steps can be merged into a single matrix \mathbf{S}$, defined as:

In [13]:
S = V*W
S = S.pinv()
pre_post = lambda X, A: transpose(X)*A*X
S

⎡1  0  0  0  0⎤
⎢             ⎥
⎢0  0  0  0  0⎥
⎢             ⎥
⎢0  0  0  0  1⎥
⎢             ⎥
⎢0  1  0  0  0⎥
⎢             ⎥
⎢0  0  1  0  0⎥
⎢             ⎥
⎣0  0  0  1  0⎦

Thus the system matrices can be redefined as follows:
### Inertia matrix

In [14]:
M = pre_post(S, M)
M

⎡M_c  0   0   0   0 ⎤
⎢                   ⎥
⎢ 0   M₁  0   0   0 ⎥
⎢                   ⎥
⎢ 0   0   M₂  0   0 ⎥
⎢                   ⎥
⎢ 0   0   0   M₃  0 ⎥
⎢                   ⎥
⎣ 0   0   0   0   Mₛ⎦

### Gyroscopic matrix

In [15]:
G = pre_post(S, G)
G

⎡G_c  0   0   0   0 ⎤
⎢                   ⎥
⎢ 0   G₁  0   0   0 ⎥
⎢                   ⎥
⎢ 0   0   G₂  0   0 ⎥
⎢                   ⎥
⎢ 0   0   0   G₃  0 ⎥
⎢                   ⎥
⎣ 0   0   0   0   Gₛ⎦

### Stiffness matrix

#### Centripetal component

In [16]:
K_Omega = pre_post(S, K_Omega)
K_Omega

⎡K_Omegac     0         0         0         0    ⎤
⎢                                                ⎥
⎢   0      K_Omega1     0         0         0    ⎥
⎢                                                ⎥
⎢   0         0      K_Omega2     0         0    ⎥
⎢                                                ⎥
⎢   0         0         0      K_Omega3     0    ⎥
⎢                                                ⎥
⎣   0         0         0         0      K_Omegas⎦

#### Bearing component:

In [17]:
K_b = pre_post(S, K_b)
K_b

⎡K_cb  0  0  0   0  ⎤
⎢                   ⎥
⎢ 0    0  0  0   0  ⎥
⎢                   ⎥
⎢ 0    0  0  0   0  ⎥
⎢                   ⎥
⎢ 0    0  0  0   0  ⎥
⎢                   ⎥
⎣ 0    0  0  0  K_sb⎦

#### Mesh component:

In [18]:
K_m = pre_post(S, K_m)
K_m

⎡  3                                                    ⎤
⎢ ___                                                   ⎥
⎢ ╲                                                     ⎥
⎢  ╲                                                    ⎥
⎢  ╱   K_c1__j  K_c2__1  K_c2__2  K_c2__3        0      ⎥
⎢ ╱                                                     ⎥
⎢ ‾‾‾                                                   ⎥
⎢j = 1                                                  ⎥
⎢                                                       ⎥
⎢   K_c2__1      K¹ₚₚ       0        0         K¹ₛ₂     ⎥
⎢                                                       ⎥
⎢   K_c2__2        0      K²ₚₚ       0         K²ₛ₂     ⎥
⎢                                                       ⎥
⎢   K_c2__3        0        0      K³ₚₚ        K³ₛ₂     ⎥
⎢                                                       ⎥
⎢                                            3          ⎥
⎢                                           ___         ⎥
⎢             

## Combine planet elements
That can be done by considering the center of mass of the $N$ planet gears, with the following matrix:

In [19]:
N1 = N_e - 1
N_c = N_e - (N - 1) # number of elements (combined planets)
C = zeros(N_c, N_e)

C[0, 0] = 1
C[N_c - 1, N_e - 1] = 1
C[1, 1:N1] = ones(1,N)*1/N
C = C.pinv()

C.pinv()*S.pinv()*q, C.pinv(), S.pinv()*q

⎛                                       ⎡q_c⎤⎞
⎜⎡    q_c     ⎤                         ⎢   ⎥⎟
⎜⎢            ⎥  ⎡1   0    0    0   0⎤  ⎢q₁ ⎥⎟
⎜⎢q₁   q₂   q₃⎥  ⎢                   ⎥  ⎢   ⎥⎟
⎜⎢── + ── + ──⎥, ⎢0  1/3  1/3  1/3  0⎥, ⎢q₂ ⎥⎟
⎜⎢3    3    3 ⎥  ⎢                   ⎥  ⎢   ⎥⎟
⎜⎢            ⎥  ⎣0   0    0    0   1⎦  ⎢q₃ ⎥⎟
⎜⎣     qₛ     ⎦                         ⎢   ⎥⎟
⎝                                       ⎣qₛ ⎦⎠

### Inertia matrix
The planets have the same mass and mass moment of inertia.

In [20]:
M_C = pre_post(C, M)
M_C = M_C.subs([(symb('M', v + 1), symb('M','')) for v in range(N)])
M_C

⎡M_c   0   0 ⎤
⎢            ⎥
⎢ 0   3⋅M  0 ⎥
⎢            ⎥
⎣ 0    0   Mₛ⎦

### Gyroscopic matrix

In [21]:
G_C = pre_post(C, G)
G_C = G_C.subs([(symb('G', v + 1), symb('G','')) for v in range(N)])
G_C

⎡G_c   0   0 ⎤
⎢            ⎥
⎢ 0   3⋅G  0 ⎥
⎢            ⎥
⎣ 0    0   Gₛ⎦

### Stiffness matrix

#### Centripetal component

In [22]:
K_Omega_C = pre_post(C, K_Omega)
K_Omega_C = K_Omega_C.subs([(symb('K', 'Omega', v + 1), symb('K', 'Omega','')) for v in range(N)])
K_Omega_C

⎡K_Omegac    0       0    ⎤
⎢                         ⎥
⎢   0      3⋅K_Ω     0    ⎥
⎢                         ⎥
⎣   0        0    K_Omegas⎦

#### Bearing component:

In [23]:
K_b_C = pre_post(C, K_b)
K_b_C

⎡K_cb  0   0  ⎤
⎢             ⎥
⎢ 0    0   0  ⎥
⎢             ⎥
⎣ 0    0  K_sb⎦

#### Mesh component

In [24]:
K_m_C = pre_post(C, K_m)
K_m_C

⎡         3                                                                  ⎤
⎢        ___                                                                 ⎥
⎢        ╲                                                                   ⎥
⎢         ╲                                                                  ⎥
⎢         ╱   K_c1__j         K_c2__1 + K_c2__2 + K_c2__3          0         ⎥
⎢        ╱                                                                   ⎥
⎢        ‾‾‾                                                                 ⎥
⎢       j = 1                                                                ⎥
⎢                                                                            ⎥
⎢K_c2__1 + K_c2__2 + K_c2__3      K¹ₚₚ + K²ₚₚ + K³ₚₚ       K¹ₛ₂ + K²ₛ₂ + K³ₛ₂⎥
⎢                                                                            ⎥
⎢                                                              3             ⎥
⎢                                                   

## Adapting it to represent a parallel gear pair
To do that, we should consider only the gear pair composed by the sun and the first planet gear. That can be done with the following matrix:

In [25]:
V = zeros(2, N_e)
V[0, 1] = 1
V[1, N_e - 1] = 1
V = V.pinv()
V.pinv()*S.pinv()*q, V.pinv(), S.pinv()*q 

⎛                       ⎡q_c⎤⎞
⎜                       ⎢   ⎥⎟
⎜                       ⎢q₁ ⎥⎟
⎜⎡q₁⎤  ⎡0  1  0  0  0⎤  ⎢   ⎥⎟
⎜⎢  ⎥, ⎢             ⎥, ⎢q₂ ⎥⎟
⎜⎣qₛ⎦  ⎣0  0  0  0  1⎦  ⎢   ⎥⎟
⎜                       ⎢q₃ ⎥⎟
⎜                       ⎢   ⎥⎟
⎝                       ⎣qₛ ⎦⎠

where the indexes 1 and $s$ now represent the wheel and pinion gears, respectively.
### Inertia matrix

In [26]:
M_par = pre_post(V, M)
M_par.subs([(symb('s',''), symb('w',''))])

⎡M₁  0 ⎤
⎢      ⎥
⎣0   Mₛ⎦

### Gyroscopic matrix

In [27]:
G_par = pre_post(V, G)
G_par

⎡G₁  0 ⎤
⎢      ⎥
⎣0   Gₛ⎦

### Stiffness matrix

#### Centripetal component

In [28]:
K_Omega_par = pre_post(V, K_Omega)
K_Omega_par

⎡K_Omega1     0    ⎤
⎢                  ⎥
⎣   0      K_Omegas⎦

#### Bearing component

In [29]:
K_b_par = pre_post(V, K_b)
K_b_par

⎡0   0  ⎤
⎢       ⎥
⎣0  K_sb⎦

In [30]:
K_m_par = pre_post(V, K_m)
K_m_par = K_m_par.as_mutable()
K_m_par[1, 1] = symb('K', 's1', '^1')
K_m_par

⎡K¹ₚₚ  K¹ₛ₂⎤
⎢          ⎥
⎣K¹ₛ₂  K¹ₛ₁⎦

Originally, the term $K_{pp}^1$ is given by:
\begin{align}
    \mathbf{K}_{pp}^n = \mathbf{K}_{c3}^n + \mathbf{K}_{r3}^n + \mathbf{K}_{s3}^n,
\end{align}
where:
* $\mathbf{K}_{c3}^n$ represents the bearing stiffness of the $n$-th planet gear
* $\mathbf{K}_{r3}^n$ represents the mesh stiffness of the $n$-th ring-planet gear pair
* $\mathbf{K}_{s3}^n$ represents the mesh stiffness of the $n$-th sun-planet gear pair

For the current case of a parallel gear pair, the term $\mathbf{K}_{r3}^n$ representing ring interactions should be excluded from $\mathbf{K}_m$. Doing that leads to:

In [31]:
K_m_par[0, 0] = symb('K', 'c3', '^1') + symb('K', 's3', '^1')
K_m_par

⎡K_c3__1 + K¹ₛ₃  K¹ₛ₂⎤
⎢                    ⎥
⎣     K¹ₛ₂       K¹ₛ₁⎦

## Submatrices of the mesh stiffness matrix $\mathbf{K}_m$
The following symplifying assumptions:
* the pressure angles of sun and ring gears are equal, i.e. $\alpha_s = \alpha_r = \alpha_n$
* the mesh stiffness for the $N$ sun-planet pairs are equal, the same happens for the $N$ ring-planet pairs
* the planetary gear is cyclically symmetric, with the $j$-th planet being at circumferential location $\psi_j$, with:
\begin{align}
    \psi_j = (j - 1) \psi, \qquad \psi = \frac{2 \pi}{N}
\end{align}

In [32]:
psi_ = lambda j: (j - 1)*symb('psi', '')
j = symbols('j', type = int)
K_c1 = lambda x: symb('k', 'p')*Matrix([[ 1           , 0           , -sin(psi_(x))],
                                        [ 0           , 1           ,  cos(psi_(x))],
                                        [-sin(psi_(x)), cos(psi_(x)),  1           ]])
k_tmp = K_c1(j)
if(not k_tmp.is_symmetric()):
    print('error.')
k_tmp

⎡        kₚ                  0          -kₚ⋅sin(ψ⋅(j - 1))⎤
⎢                                                         ⎥
⎢        0                  kₚ          kₚ⋅cos(ψ⋅(j - 1)) ⎥
⎢                                                         ⎥
⎣-kₚ⋅sin(ψ⋅(j - 1))  kₚ⋅cos(ψ⋅(j - 1))          kₚ        ⎦

where $k_p$ is the planet bearing stiffness.

In [33]:
K_c2 = lambda x: symb('k', 'p')*Matrix([[-cos(psi_(x)),  sin(psi_(x)), 0],
                                        [-sin(psi_(x)), -cos(psi_(x)), 0],
                                        [ 0           , -1           , 0]])
k_tmp = K_c2(j)
k_tmp

⎡-kₚ⋅cos(ψ⋅(j - 1))  kₚ⋅sin(ψ⋅(j - 1))   0⎤
⎢                                         ⎥
⎢-kₚ⋅sin(ψ⋅(j - 1))  -kₚ⋅cos(ψ⋅(j - 1))  0⎥
⎢                                         ⎥
⎣        0                  -kₚ          0⎦

In [34]:
K_c3 = symb('k', 'p')*diag(1, 1, 0)
K_c3

⎡kₚ  0   0⎤
⎢         ⎥
⎢0   kₚ  0⎥
⎢         ⎥
⎣0   0   0⎦

Notice that the planet bearings are isotropic, presenting the same stiffness in directions $x$ and $y$, different from the other bearing stiffness submatrices that present different $x$, $y$, and $u$ components. One can rewrite the matrices $\mathbf{K}_{ci}^n$ for anysotropic bearings in the following way:
\begin{align}
    \mathbf{K}_{ci}^n(\mathrm{new}) = \frac{1}{k_{pj}}\mathbf{K}_{ci}^n(\mathrm{old}) \mathrm{diag}(k_{px}, k_{py}, k_{pu})
\end{align}

In [35]:
def K_c(i, j):
    if(i == 1):
        k_old = K_c1(j)
    elif(i == 2):
        k_old = K_c2(j)
    elif(i == 3):
        k_old = K_c3
    else:
        print('ERROR inf K_c(), i = {} is out of range.'.format(i))
    
    return (1/symb('k', 'p'))*diag(symb('k', 'p', 'x'), symb('k', 'p', 'y'), 0*symb('k', 'p', 'u'))*k_old
    
K_c(1, j), K_c(2, j), K_c(3, j)

⎛⎡kₚₓ   0    -kₚₓ⋅sin(ψ⋅(j - 1))⎤  ⎡-kₚₓ⋅cos(ψ⋅(j - 1))    kₚₓ⋅sin(ψ⋅(j - 1)) 
⎜⎢                              ⎥  ⎢                                          
⎜⎢ 0   k_py  k_py⋅cos(ψ⋅(j - 1))⎥, ⎢-k_py⋅sin(ψ⋅(j - 1))  -k_py⋅cos(ψ⋅(j - 1))
⎜⎢                              ⎥  ⎢                                          
⎝⎣ 0    0             0         ⎦  ⎣         0                     0          

  0⎤  ⎡kₚₓ   0    0⎤⎞
   ⎥  ⎢            ⎥⎟
  0⎥, ⎢ 0   k_py  0⎥⎟
   ⎥  ⎢            ⎥⎟
  0⎦  ⎣ 0    0    0⎦⎠

In [36]:
psi, alpha_n = symbols('psi alpha_n', type = float)
psi_s = lambda x: psi_(x) - alpha_n
psi_r = lambda x: psi_(x) + alpha_n

K_r1 = lambda x: symb('k', 'r')*Matrix([[               sin(psi_r(x))**2, -cos(psi_r(x))*sin(psi_r(x)), -sin(psi_r(x))],
                                        [-cos(psi_r(x))*sin(psi_r(x))   ,  cos(psi_r(x))**2           ,  cos(psi_r(x))],
                                        [-              sin(psi_r(x))   ,  cos(psi_r(x))              ,  1]])
k_tmp = K_r1(j)
if(not k_tmp.is_symmetric()):
    print('error.')
k_tmp

⎡                2                                                            
⎢          kᵣ⋅sin (αₙ + ψ⋅(j - 1))            -kᵣ⋅sin(αₙ + ψ⋅(j - 1))⋅cos(αₙ +
⎢                                                                             
⎢                                                             2               
⎢-kᵣ⋅sin(αₙ + ψ⋅(j - 1))⋅cos(αₙ + ψ⋅(j - 1))            kᵣ⋅cos (αₙ + ψ⋅(j - 1)
⎢                                                                             
⎣          -kᵣ⋅sin(αₙ + ψ⋅(j - 1))                      kᵣ⋅cos(αₙ + ψ⋅(j - 1))

                                    ⎤
 ψ⋅(j - 1))  -kᵣ⋅sin(αₙ + ψ⋅(j - 1))⎥
                                    ⎥
                                    ⎥
)            kᵣ⋅cos(αₙ + ψ⋅(j - 1)) ⎥
                                    ⎥
                       kᵣ           ⎦

where $k_{r}$ is the mesh stiffness between ring and planet gears.

In [37]:
K_r2 = lambda x: symb('k', 'r')*Matrix([[-sin(psi_r(x))*sin(alpha_n),  sin(psi_r(x))*cos(alpha_n),  sin(psi_r(x))],
                                        [ cos(psi_r(x))*sin(alpha_n), -cos(psi_r(x))*cos(alpha_n), -cos(psi_r(x))],
                                        [               sin(alpha_n), -              cos(alpha_n), -1]])
k_tmp = K_r2(j)
k_tmp

⎡-kᵣ⋅sin(αₙ)⋅sin(αₙ + ψ⋅(j - 1))  kᵣ⋅sin(αₙ + ψ⋅(j - 1))⋅cos(αₙ)   kᵣ⋅sin(αₙ +
⎢                                                                             
⎢kᵣ⋅sin(αₙ)⋅cos(αₙ + ψ⋅(j - 1))   -kᵣ⋅cos(αₙ)⋅cos(αₙ + ψ⋅(j - 1))  -kᵣ⋅cos(αₙ 
⎢                                                                             
⎣          kᵣ⋅sin(αₙ)                       -kᵣ⋅cos(αₙ)                      -

 ψ⋅(j - 1)) ⎤
            ⎥
+ ψ⋅(j - 1))⎥
            ⎥
kᵣ          ⎦

In [38]:
K_r3 = lambda x: symb('k', 'r')*Matrix([[ sin(alpha_n)**2          , -sin(alpha_n)*cos(alpha_n), -sin(alpha_n)],
                                        [-sin(alpha_n)*cos(alpha_n),  cos(alpha_n)**2          ,  cos(alpha_n)],
                                        [-sin(alpha_n)             ,  cos(alpha_n)             ,  1]])
k_tmp = K_r3('j')
if(not k_tmp.is_symmetric()):
    print('error.')
k_tmp

⎡          2                                          ⎤
⎢    kᵣ⋅sin (αₙ)      -kᵣ⋅sin(αₙ)⋅cos(αₙ)  -kᵣ⋅sin(αₙ)⎥
⎢                                                     ⎥
⎢                               2                     ⎥
⎢-kᵣ⋅sin(αₙ)⋅cos(αₙ)      kᵣ⋅cos (αₙ)      kᵣ⋅cos(αₙ) ⎥
⎢                                                     ⎥
⎣    -kᵣ⋅sin(αₙ)          kᵣ⋅cos(αₙ)           kᵣ     ⎦

In [39]:
K_s1 = lambda x: symb('k', 's')*Matrix([[               sin(psi_s(x))**2, -cos(psi_s(x))*sin(psi_s(x)), -sin(psi_s(x))],
                                        [-cos(psi_s(x))*sin(psi_s(x))   ,  cos(psi_s(x))**2           ,  cos(psi_s(x))],
                                        [-              sin(psi_s(x))   ,  cos(psi_s(x))              ,  1]])
k_tmp = K_s1(j)
if(not k_tmp.is_symmetric()):
    print('error.')
k_tmp

⎡               2                                                             
⎢         kₛ⋅sin (αₙ - ψ⋅(j - 1))            kₛ⋅sin(αₙ - ψ⋅(j - 1))⋅cos(αₙ - ψ
⎢                                                                             
⎢                                                           2                 
⎢kₛ⋅sin(αₙ - ψ⋅(j - 1))⋅cos(αₙ - ψ⋅(j - 1))           kₛ⋅cos (αₙ - ψ⋅(j - 1)) 
⎢                                                                             
⎣          kₛ⋅sin(αₙ - ψ⋅(j - 1))                      kₛ⋅cos(αₙ - ψ⋅(j - 1)) 

                                 ⎤
⋅(j - 1))  kₛ⋅sin(αₙ - ψ⋅(j - 1))⎥
                                 ⎥
                                 ⎥
           kₛ⋅cos(αₙ - ψ⋅(j - 1))⎥
                                 ⎥
                     kₛ          ⎦

where $k_{sj}$ is the mesh stiffness between the sun and the $j$-th planet

In [40]:
K_s2 = lambda x: symb('k', 's')*Matrix([[ sin(psi_s(x))*sin(alpha_n),  sin(psi_s(x))*cos(alpha_n), -sin(psi_s(x))],
                                        [-cos(psi_s(x))*sin(alpha_n), -cos(psi_s(x))*cos(alpha_n),  cos(psi_s(x))],
                                        [-              sin(alpha_n), -              cos(alpha_n),  1]])
K_s2(j)

⎡-kₛ⋅sin(αₙ)⋅sin(αₙ - ψ⋅(j - 1))  -kₛ⋅sin(αₙ - ψ⋅(j - 1))⋅cos(αₙ)  kₛ⋅sin(αₙ -
⎢                                                                             
⎢-kₛ⋅sin(αₙ)⋅cos(αₙ - ψ⋅(j - 1))  -kₛ⋅cos(αₙ)⋅cos(αₙ - ψ⋅(j - 1))  kₛ⋅cos(αₙ -
⎢                                                                             
⎣          -kₛ⋅sin(αₙ)                      -kₛ⋅cos(αₙ)                      k

 ψ⋅(j - 1))⎤
           ⎥
 ψ⋅(j - 1))⎥
           ⎥
ₛ          ⎦

In [41]:
K_s3 = lambda x: symb('k', 's')*Matrix([[ sin(alpha_n)**2          ,  sin(alpha_n)*cos(alpha_n), -sin(alpha_n)],
                                        [ sin(alpha_n)*cos(alpha_n),  cos(alpha_n)**2          , -cos(alpha_n)],
                                        [-sin(alpha_n)             , -cos(alpha_n)             ,  1]])
k_tmp = K_s3(j)
if(not k_tmp.is_symmetric()):
    print('error.')
k_tmp

⎡         2                                         ⎤
⎢   kₛ⋅sin (αₙ)      kₛ⋅sin(αₙ)⋅cos(αₙ)  -kₛ⋅sin(αₙ)⎥
⎢                                                   ⎥
⎢                             2                     ⎥
⎢kₛ⋅sin(αₙ)⋅cos(αₙ)     kₛ⋅cos (αₙ)      -kₛ⋅cos(αₙ)⎥
⎢                                                   ⎥
⎣   -kₛ⋅sin(αₙ)         -kₛ⋅cos(αₙ)          kₛ     ⎦

In [42]:
K_pp = lambda x: K_c3 + K_r3(x) + K_s3(x)
k_tmp = K_pp(j)
if(not k_tmp.is_symmetric()):
    print('error.')
k_tmp

⎡                2             2                                              
⎢     kₚ + kᵣ⋅sin (αₙ) + kₛ⋅sin (αₙ)       -kᵣ⋅sin(αₙ)⋅cos(αₙ) + kₛ⋅sin(αₙ)⋅co
⎢                                                                             
⎢                                                          2             2    
⎢-kᵣ⋅sin(αₙ)⋅cos(αₙ) + kₛ⋅sin(αₙ)⋅cos(αₙ)       kₚ + kᵣ⋅cos (αₙ) + kₛ⋅cos (αₙ)
⎢                                                                             
⎣        -kᵣ⋅sin(αₙ) - kₛ⋅sin(αₙ)                  kᵣ⋅cos(αₙ) - kₛ⋅cos(αₙ)    

                               ⎤
s(αₙ)  -kᵣ⋅sin(αₙ) - kₛ⋅sin(αₙ)⎥
                               ⎥
                               ⎥
       kᵣ⋅cos(αₙ) - kₛ⋅cos(αₙ) ⎥
                               ⎥
               kᵣ + kₛ         ⎦

## Rotational coordinates
The rotation coordinates are:
\begin{align}
u_j = r_j \theta_j,
\end{align}
where $\theta_j$ and $r_j$ are the component rotation and the base radius of a component, respectively. For the planet carrier, one should use the center distance instead. One can write the equations of motion in terms of the component rotations, by using the following matrix:

In [43]:
R = lambda x: diag(1, 1, symb('r', x))
R(j)

⎡1  0   0 ⎤
⎢         ⎥
⎢0  1   0 ⎥
⎢         ⎥
⎣0  0  r_j⎦

### Inertia matrix

In [44]:
pre_post(R(j), M_(j))

⎡m_j   0    0 ⎤
⎢             ⎥
⎢ 0   m_j   0 ⎥
⎢             ⎥
⎣ 0    0   I_j⎦

This operation does not affect the gyroscopic and centripetal submatrices $\mathbf{G}_j$ and $\mathbf{K}_{\Omega,j}$.

### Stiffness matrices

#### Centripetal component

Not affected.

#### Bearing component

In [45]:
pre_post(R(j), K_b_(j))

⎡k_jx   0        0    ⎤
⎢                     ⎥
⎢ 0    k_jy      0    ⎥
⎢                     ⎥
⎢                    2⎥
⎣ 0     0    k_ju⋅r_j ⎦

#### Mesh component